# DL_Final_Model

## Knowing when to look

### LSTM part 

In [1]:
import torch.nn.functional as F

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AdaptiveAttentionLstm(nn.Module):
    def __init__(self, opt, maxout=True):
        self.att_feat_size = opt.att_feat_size
        self.dropout = opt.dropout  
        self.maxout = opt.maxout
        self.training = opt.training
        self.input_size_L = opt.input_size_L
        
        #Build the LSTM network
        self.w2h = nn.Linear(self.input_encoding_size, (4+(use_maxout==True)) * self.rnn_size)
        self.v2h = nn.Linear(self.rnn_size, (4+(use_maxout==True)) * self.rnn_size)
        
        self.i2h = nn.ModuleList([nn.Linear(self.rnn_size, (4+(use_maxout==True)) * self.rnn_size) for _ in range(self.num_layers - 1)])
        self.h2h = nn.ModuleList([nn.Linear(self.rnn_size, (4+(use_maxout==True)) * self.rnn_size) for _ in range(self.num_layers)])
        
        
            
    def forward(self, x_t, state):
        # hidden states of all layers
        hidden_states = []
        # cell states of all layers
        cell_states = []
        for layer in range(self.num_layers):
            # hidden state of previous timestamp
            prev_h = state[0][layer]
            # cell state of previous timestamp
            prev_c = state[1][layer]
            
            if layer == 0: 
                x = x_t
                i2h = self.w2h(x)
            else:
                x = hidden_states[-1]
                x = F.dropout(x, p=self.dropout, training=self.training)
                i2h = self.i2h[layer-1](x)
            
            all_input_sums = i2h + self.h2h[layer](prev_h)
        

local LSTM = {}
function LSTM.lstm(input_size, rnn_size, n, dropout)
    dropout = dropout or 0 

# there will be 2*n+1 inputs
    local inputs = {}
    table.insert(inputs, nn.Identity()()) # indices giving the sequence of symbols
    table.insert(inputs, nn.Identity()()) # indices giving the image feature  
    for L = 1,n:
        table.insert(inputs, nn.Identity()()) # prev_c[L]
        table.insert(inputs, nn.Identity()()) # prev_h[L]

    local img_fc = inputs[2]

    local x, input_size_L, i2h, fake_region, atten_region
    local outputs = {}
    for L = 1,n do
        # c,h from previos timesteps
        local prev_h = inputs[L*2+2]
        local prev_c = inputs[L*2+1]
        # the input to this layer
        if L == 1 then 
            x = inputs[1]
            input_size_L = input_size
            local w2h = nn.Linear(input_size_L, 4 * rnn_size)(x):annotate{name='t2h_'..L}
            local v2h = nn.Linear(input_size_L, 4 * rnn_size)(img_fc):annotate{name='v2h_'..L}
            i2h = nn.CAddTable()({w2h, v2h})
        else
            x = outputs[(L-1)*2]
            if dropout > 0 then x = nn.Dropout(dropout)(x):annotate{name='drop_' .. L} end -- apply dropout, if any
            input_size_L = rnn_size
            i2h = nn.Linear(input_size_L, 4 * rnn_size)(x):annotate{name='i2h_'..L}
        end

        local h2h = nn.Linear(rnn_size, 4 * rnn_size)(prev_h):annotate{name='h2h_'..L}
        local all_input_sums = nn.CAddTable()({i2h, h2h})

        local reshaped = nn.Reshape(4, rnn_size)(all_input_sums)
        local n1, n2, n3, n4 = nn.SplitTable(2)(reshaped):split(4)
        # decode the gates
        local in_gate = nn.Sigmoid()(n1)
        local forget_gate = nn.Sigmoid()(n2)
        local out_gate = nn.Sigmoid()(n3)
        # decode the write inputs
        local in_transform = nn.Tanh()(n4)
        # perform the LSTM update
        local next_c           = nn.CAddTable()({
            nn.CMulTable()({forget_gate, prev_c}),
            nn.CMulTable()({in_gate,     in_transform})
          })

        local tanh_nex_c = nn.Tanh()(next_c)
        # gated cells form the output
        local next_h = nn.CMulTable()({out_gate,tanh_nex_c})
        if L == n then
            if L==1 then
                local w2h = nn.Linear(input_size_L, 1 * rnn_size)(x)
                local v2h = nn.Linear(input_size_L, 1 * rnn_size)(img_fc) 
                i2h = nn.CAddTable()({w2h, v2h})
            else
                i2h = nn.Linear(input_size_L, rnn_size)(x)
            end      
            local h2h = nn.Linear(rnn_size, rnn_size)(prev_h)
            local n5 = nn.CAddTable()({i2h, h2h})

            fake_region = nn.CMulTable()({nn.Sigmoid()(n5), tanh_nex_c})
        end

        table.insert(outputs, next_c)
        table.insert(outputs, next_h)
    end
    # set up the decoder
    local top_h = nn.Identity()(outputs[#outputs])
    if dropout > 0 then top_h = nn.Dropout(dropout)(top_h) end
    if dropout > 0 then fake_region = nn.Dropout(dropout)(fake_region) end

    table.insert(outputs, top_h)
    table.insert(outputs, fake_region)
    return nn.gModule(inputs, outputs)
end

return LSTM


### Adaptive attention model

In [ ]:
class AdaptiveAttention(CaptionModel):
    def __init__(self, opt):
        super(AdaptiveAttention, self).__init__()
        self.vocab_size = opt.vocab_size
        self.input_encoding_size = opt.input_encoding_size       # 512
        self.rnn_size = opt.rnn_size       # 512
        self.num_layers = opt.num_layers   # 1
        self.dropout = opt.dropout         # 0.5
        self.seq_length = opt.seq_length   # max length for the caption 20
        self.fc_size = opt.fc_size         # 2048
        self.conv_size = opt.conv_size     # 2048
        self.hidden_size = opt.hidden_size #
        
        self.image_embedding = 
            
            
            
            